In [2]:
import os
from BiasedErasure.delayed_erasure_decoders.Experimental_Loss_Decoder import *
import time
import numpy as np
from statsmodels.stats.proportion import proportion_confint


distance = 5
output_dir = '/Users/gefenbaranes/Documents/CX_experiment'

decoder_basis = 'XX'
gate_ordering = ['N', 'Z']
num_ancilla_blocks = 4
num_rounds = num_ancilla_blocks + 1
# measurement_events = np.load(f'/Users/gefenbaranes/Lukin SiV Dropbox/Gefen Baranes/local_Repetitive_QEC/Oct24/measurement_events_X_NZZrNr_2024_10_06.npy')
exp_measurements = np.load(f'/Users/gefenbaranes/Documents/2024_10_15_measurement_events_1CNOT_XX.npy')
measurement_events = np.concatenate([exp_measurements[:, 0, :distance**2-1],
                            exp_measurements[:, 1, :distance**2-1],
                            exp_measurements[:, 0, distance**2-1:2*(distance**2-1)],
                            exp_measurements[:, 1, distance**2-1:2*(distance**2-1)],
                            exp_measurements[:, 0, 2*(distance**2-1):],
                            exp_measurements[:, 1, 2*(distance**2-1):]], axis=1)



detection_event_signs = np.load(f'/Users/gefenbaranes/Documents/detection_event_signs_2024_10_15.npy')
print(measurement_events)

num_of_shots = len(measurement_events)

train_measurement_events = measurement_events[::2]
test_measurement_events = measurement_events[1::2]

measurement_events = measurement_events[:100]
# measurement_events = test_measurement_events

print(f"len(measurement_events) = {len(measurement_events)}")

noise_params = {}

Meta_params = {'architecture': 'CBQC', 'code': 'Rotated_Surface', 'logical_basis': decoder_basis,
            'bias_preserving_gates': 'False',
            'noise': 'atom_array', 'is_erasure_biased': 'False', 'LD_freq': '1000', 'LD_method': 'None',
            'SSR': 'True', 'cycles': str(num_rounds - 1),
            'ordering': gate_ordering,
            'decoder': 'MLE',
            'circuit_type': 'logical_CX_3', 'Steane_type': 'None', 'printing': 'False', 'num_logicals': '2',
            'loss_decoder': 'independent',
            'obs_pos': 'd-1', 'n_r': '0'}

print(gate_ordering)

use_loss_decoding = True  # if False: use same DEM every shot, without utilizing SSR.
use_independent_decoder = True  # if False: in every lifecycle, we just apply supercheck at the end. If True: we count the full lifecycle with different potential loss locations and corresponding Clifford propagations.
use_independent_and_first_comb_decoder = False  # This is relevant only if use_independent_decoder=True. If False: use only independent lifecycles. If True: adds a single combination of lifecycles to the decoder.


simulate_data = True

num_shots = 100 # Change #1: if simulate_data = True, you can set num_shots, and put as input measurement_events = None
predictions, observable_flips, dems_list = Loss_MLE_Decoder_Experiment(Meta_params, distance, distance, output_dir,
                                                                    measurement_events,
                                                                    detection_event_signs, use_loss_decoding,
                                                                    use_independent_decoder,
                                                                    use_independent_and_first_comb_decoder,
                                                                    simulate_data=simulate_data, logical_gaps=False, 
                                                                    noise_params=noise_params, num_shots=num_shots)





logical_probability = np.mean(np.logical_xor(observable_flips, predictions))

# logical_probability = np.mean(np.logical_xor(predictions[:, 0], observable_flips.flatten())).squeeze()
print(r'Probability of logical +1: %.6f' % (1 - logical_probability))

[[0 0 0 ... 0 1 0]
 [0 1 0 ... 0 2 0]
 [0 0 0 ... 1 1 1]
 ...
 [1 1 1 ... 2 1 0]
 [1 1 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]]
len(measurement_events) = 100
['N', 'Z']
num_CX_per_layer = 3
final measurement_index = 146
0 
Interrupt request received
Probability of logical +1: 0.950000


In [4]:
num_shots = 10
# Change #2: a function that doesn't decode, just gives you the measurements, detection events, observables and the circuit:
measurement_events_all_shots, detection_events_all_shots, observable_flips_all_shots, LogicalCircuit = get_simulated_measurement_events(Meta_params, distance, distance, num_shots, noise_params)

num_CX_per_layer = 2
Using orderings: ['N', 'Z']
sampling for the following loss pattern: [   6  859  937  959  988 1138 1528 2153 3061]
{2: [13], 96: [19, 97], 104: [21, 50], 113: [4], 133: [56], 182: [89], 262: [67]}
dem = error(0.006265333524878385213) D0
error(0.000731117684804259726) D0 D1 D27 D28
error(0.0003745640403358636373) D0 D1 D28
error(0.001105158136830374687) D0 D3
error(0.0004795085878160105109) D0 D4 D27 D33 D72
error(0.5) D0 D4 D33 D72
error(0.0007214161525332775729) D0 D6 D84
error(0.0003842964596437939005) D0 D6 D84 D97
error(0.002870930643198536369) D0 D7 D17 D18 D84
error(0.000365692573460413956) D0 D7 D17 D18 D84 D97
error(0.002589018318443744686) D0 D7 D84
error(1.860542025217774899e-05) D0 D7 D84 D97
error(0.003235680414864576179) D0 D12
error(0.003962983486122255081) D0 D24
error(0.003350114741890674284) D0 D24 D26
error(0.07080094136218299805) D0 D26
error(0.00137462722593160243) D0 D26 D32
error(0.00211700819407830863) D0 D27
error(0.003235680414864576179) D

In [16]:
distance = 5
# qubit_states_nans_perlog = np.load('2024_10_15_measurement_events_1CNOT_XX.npy').astype(bool)
qubit_states_nans_perlog = np.load(f'/Users/gefenbaranes/Documents/2024_10_15_measurement_events_1CNOT_XX.npy')
qubit_states_nans_perlog = qubit_states_nans_perlog.transpose(1,2,0)
# exp_measurements = np.concatenate([exp_measurements[:, 0, :distance**2-1],
#                             exp_measurements[:, 1, :distance**2-1],
#                             exp_measurements[:, 0, distance**2-1:2*(distance**2-1)],
#                             exp_measurements[:, 1, distance**2-1:2*(distance**2-1)],
#                             exp_measurements[:, 0, 2*(distance**2-1):],
#                             exp_measurements[:, 1, 2*(distance**2-1):]], axis=1)


perfect_reps = np.ones(qubit_states_nans_perlog.shape[-1]).astype(bool)

In [17]:

num_data_blocks = num_logicals = 2

num_ancilla_blocks = 2
num_rounds = num_ancilla_blocks + 1

ancilla_grid_size = 6
data_grid_size = d = 5
xspc = 3
yspc = 2

num_datas = 25
num_ancillas = 24
total_num_ancillas = num_ancillas*num_ancilla_blocks*num_logicals
total_num_datas = num_datas*num_data_blocks
num_physicals = total_num_ancillas + total_num_datas



stabilizer_weights = np.zeros(num_ancillas)
stabilizer_masks = np.zeros((num_ancillas, num_datas), dtype = bool)

ancilla_Astabs_mask = Zstabs_mask =  np.array([1,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,1], dtype = bool) 
ancilla_Bstabs_mask = Xstabs_mask = (1-ancilla_Astabs_mask).astype(bool)

deterministic_rounds = np.ones(num_rounds).astype(bool)
nondeterministic_rounds = np.array([0] + (num_rounds - 2)*[1] + [0]).astype(bool)


### HARDCODE FOR NOW
stabilizer_weights = np.array([2., 2., 4., 4., 4., 4., 2., 2., 4., 4., 4., 4., 4., 4., 4., 4., 2.,
       2., 4., 4., 4., 4., 2., 2.])
stabilizer_masks = np.array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]).astype(bool)


stab2A = (ancilla_Astabs_mask & (stabilizer_weights == 2))
stab2B = (ancilla_Bstabs_mask & (stabilizer_weights == 2))
stab4A = (ancilla_Astabs_mask & (stabilizer_weights == 4))
stab4B = (ancilla_Bstabs_mask & (stabilizer_weights == 4))


vertical_string_masks = np.zeros((d,num_datas)).astype(bool)
horizontal_string_masks = np.zeros((d,num_datas)).astype(bool)

for i in range(d):
    vertical_string_masks[i,i::d] = True
    horizontal_string_masks[i,i*d:d*(i+1)] = True

# qubit_states_nans = atoms_present_final.copy()


# ## Reshape into per logical
# qubit_states_nans_perlog = np.array([qubit_states_nans[logical_qubit_masks[i]] for i in range(num_logicals)])
# qubit_states_perlog = np.nan_to_num(qubit_states_nans_perlog, nan = 3) #convert nan to 3 (so it then becomes 2 in the next line)
# qubit_states_perlog = ((qubit_states_perlog + 1)/2).astype(int)

# data_losses_perlog = np.isnan(qubit_states_nans_perlog[:,-num_datas:])
# data_losses_per_stab_perlog = np.array([np.sum(data_losses_perlog[:,stabilizer_masks[i]], axis = 1) for i in range(num_ancillas)]).transpose(1,0,2)
# num_data_losses_perlog = np.sum(data_losses_perlog, axis = 1)




stabilizer_prod_per_round = []
stabilizer_prod_per_round_withloss = []

loss_sign = +1

for r in range(-1,num_ancilla_blocks):
    if r == -1:
        stab_second = qubit_states_nans_perlog[:,num_ancillas*(r+1):num_ancillas*(r+2)]
        stab_first = np.ones_like(stab_second)
        stab_first_withloss = np.nan_to_num(stab_first,nan = loss_sign)
        stab_second_withloss = np.nan_to_num(stab_second,nan = loss_sign)
                                            
    elif r == num_ancilla_blocks - 1:
        stab_first = qubit_states_nans_perlog[:,num_ancillas*r:num_ancillas*(r+1)]
        stab_first_withloss = np.nan_to_num(stab_first,nan = loss_sign)
        stab_second = np.array([np.prod(qubit_states_nans_perlog[:,-num_datas:][:,stabilizer_masks[i]],axis = 1) for i in range(num_ancillas)]).transpose(1,0,2)
        stab_second_withloss = np.array([np.prod(np.nan_to_num(qubit_states_nans_perlog[:,-num_datas:][:,stabilizer_masks[i]], nan = loss_sign),axis = 1) for i in range(num_ancillas)]).transpose(1,0,2)
    else:
        stab_first = qubit_states_nans_perlog[:,num_ancillas*r:num_ancillas*(r+1)]
        stab_first_withloss = np.nan_to_num(stab_first,nan = loss_sign)
        stab_second = qubit_states_nans_perlog[:,num_ancillas*(r+1):num_ancillas*(r+2)]
        stab_second_withloss = np.nan_to_num(stab_second, nan = loss_sign)

    stab_product_per_round = stab_first*stab_second
    stab_product_per_round_withloss = stab_first_withloss*stab_second_withloss #convert loss to qubit state |0>

    stabilizer_prod_per_round.append(stab_product_per_round)
    stabilizer_prod_per_round_withloss.append(stab_product_per_round_withloss)

stabilizer_prod_per_round = np.array(stabilizer_prod_per_round)
stabilizer_prod_per_round_withloss = np.array(stabilizer_prod_per_round_withloss)

# For sublattice A, and sublattice B separately

error_prob_Astabs = 1-(1+np.abs(np.nanmean(stabilizer_prod_per_round[:,:,ancilla_Astabs_mask],axis = -1)))/2
error_prob_Bstabs = 1-(1+np.abs(np.nanmean(stabilizer_prod_per_round[:,:,ancilla_Bstabs_mask], axis= -1)))/2

error_prob_Astabs_postselected = 1-(1+np.abs(np.nanmean(stabilizer_prod_per_round[:,:,ancilla_Astabs_mask][:,:,:,perfect_reps],axis = -1)))/2
error_prob_Bstabs_postselected = 1-(1+np.abs(np.nanmean(stabilizer_prod_per_round[:,:,ancilla_Bstabs_mask][:,:,:,perfect_reps], axis= -1)))/2

error_prob_Astabs_fully_postselected = np.zeros_like(error_prob_Astabs_postselected)
error_prob_Bstabs_fully_postselected = np.zeros_like(error_prob_Bstabs_postselected)

stabilizer_prod_per_round_per_stab_dataloss_postselection = []
for log in range(num_logicals):
    stabilizer_prod_per_round_per_stab_dataloss_postselection += [[stabilizer_prod_per_round[:,log,i][:,data_losses_per_stab_perlog[log,i] == 0][:,perfect_reps[data_losses_per_stab_perlog[log,i] == 0]] for i in range(num_ancillas)]]
    error_prob_Astabs_fully_postselected[:,log] = 1-(1+np.abs([np.nanmean(stabilizer_prod_per_round_per_stab_dataloss_postselection[-1][i],axis = -1) for i in np.where(ancilla_Astabs_mask)[0]]).T)/2
    error_prob_Bstabs_fully_postselected[:,log] = 1-(1+np.abs([np.nanmean(stabilizer_prod_per_round_per_stab_dataloss_postselection[-1][i],axis = -1) for i in np.where(ancilla_Bstabs_mask)[0]]).T)/2

plt.figure("Detected error per round", figsize = (15,5))
plt.clf()
for log in range(num_logicals):  
    Astab_good_rounds_thislog = deterministic_rounds if prep_basis[log] == 'vertical' else nondeterministic_rounds
    Bstab_good_rounds_thislog = nondeterministic_rounds if prep_basis[log] == 'vertical' else deterministic_rounds
    
    plt.subplot(1,2,log+1)
    plt.bar(np.arange(num_rounds)-0.2,np.nanmean(error_prob_Astabs[:,log],axis = -1),width = 0.4,label = "A", color = "b", alpha = 0.6)
    plt.bar(np.arange(num_rounds)+0.2,np.nanmean(error_prob_Bstabs[:,log],axis = -1),width = 0.4,label = "B", color = "orange", alpha = 0.6)
    plt.bar(np.arange(num_rounds)-0.2,np.nanmean(error_prob_Astabs_postselected[:,log],axis = -1),width = 0.4,label = "A (post)",color = "b")
    plt.bar(np.arange(num_rounds)+0.2,np.nanmean(error_prob_Bstabs_postselected[:,log],axis = -1),width = 0.4,label = "B (post)", color = "orange")
    
    plt.xticks(np.arange(num_rounds))
    plt.xlabel("Round")
    plt.ylabel("Detected error probability")
    plt.ylim(0,0.6)
    plt.legend()
    plt.title(f"""Average error prob (no postselection, exclude loss): {np.nanmean(np.concatenate((error_prob_Astabs[:,log][Astab_good_rounds_thislog],error_prob_Bstabs[:,log][Bstab_good_rounds_thislog]))):.4f}
    Average error prob (postselected, exclude loss): {np.nanmean(np.concatenate((error_prob_Astabs_postselected[:,log][Astab_good_rounds_thislog],error_prob_Bstabs_postselected[:,log][Bstab_good_rounds_thislog]))):.4f}""");

NameError: name 'data_losses_per_stab_perlog' is not defined

In [9]:
error_prob_Astabs_postselected_weight_2 = 1-(1+np.abs(np.nanmean(stabilizer_prod_per_round[:,:,stab2A][:,:,:,perfect_reps],axis = -1)))/2
error_prob_Bstabs_postselected_weight_2  = 1-(1+np.abs(np.nanmean(stabilizer_prod_per_round[:,:,stab2B][:,:,:,perfect_reps], axis= -1)))/2
error_prob_Astabs_postselected_weight_4 = 1-(1+np.abs(np.nanmean(stabilizer_prod_per_round[:,:,stab4A][:,:,:,perfect_reps],axis = -1)))/2
error_prob_Bstabs_postselected_weight_4  = 1-(1+np.abs(np.nanmean(stabilizer_prod_per_round[:,:,stab4B][:,:,:,perfect_reps], axis= -1)))/2

error_prob_Astabs_postselected_weight_2_err = np.nanstd(stabilizer_prod_per_round[:,:,stab2A][:,:,:,perfect_reps],axis = -1)/2/np.sqrt(stabilizer_prod_per_round[:,:,stab2A][:,:,:,perfect_reps].shape[-1])
error_prob_Bstabs_postselected_weight_2_err  = np.nanstd(stabilizer_prod_per_round[:,:,stab2B][:,:,:,perfect_reps], axis= -1)/2/np.sqrt(stabilizer_prod_per_round[:,:,stab2B][:,:,:,perfect_reps].shape[-1])
error_prob_Astabs_postselected_weight_4_err = np.nanstd(stabilizer_prod_per_round[:,:,stab4A][:,:,:,perfect_reps],axis = -1)/2/np.sqrt(stabilizer_prod_per_round[:,:,stab4A][:,:,:,perfect_reps].shape[-1])
error_prob_Bstabs_postselected_weight_4_err  = np.nanstd(stabilizer_prod_per_round[:,:,stab4B][:,:,:,perfect_reps], axis= -1)/2/np.sqrt(stabilizer_prod_per_round[:,:,stab4B][:,:,:,perfect_reps].shape[-1])

stab_mean_prod_stab2A = np.array([np.abs([np.nanmean(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i],axis = -1) for i in np.where(stab2A)[0]]).T for log in range(num_logicals)]).transpose(1,0,2)
stab_mean_prod_stab2B = np.array([np.abs([np.nanmean(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i],axis = -1) for i in np.where(stab2B)[0]]).T for log in range(num_logicals)]).transpose(1,0,2)
stab_mean_prod_stab4A = np.array([np.abs([np.nanmean(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i],axis = -1) for i in np.where(stab4A)[0]]).T for log in range(num_logicals)]).transpose(1,0,2)
stab_mean_prod_stab4B = np.array([np.abs([np.nanmean(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i],axis = -1) for i in np.where(stab4B)[0]]).T for log in range(num_logicals)]).transpose(1,0,2)

error_prob_Astabs_fully_postselected_weight_2 = 1-(1+np.abs(stab_mean_prod_stab2A))/2
error_prob_Bstabs_fully_postselected_weight_2  = 1-(1+np.abs(stab_mean_prod_stab2B))/2
error_prob_Astabs_fully_postselected_weight_4 = 1-(1+np.abs(stab_mean_prod_stab4A))/2
error_prob_Bstabs_fully_postselected_weight_4  = 1-(1+np.abs(stab_mean_prod_stab4B))/2

error_prob_Astabs_fully_postselected_weight_2_err = np.array([0.5 * np.array([np.nanstd(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i],axis = -1)/np.sqrt(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i].shape[-1]) 
                                                                              for i in np.where(stab2A)[0]]).T for log in range(num_logicals)]).transpose(1,0,2)
error_prob_Bstabs_fully_postselected_weight_2_err = np.array([0.5 * np.array([np.nanstd(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i],axis = -1)/np.sqrt(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i].shape[-1]) 
                                                                              for i in np.where(stab2B)[0]]).T for log in range(num_logicals)]).transpose(1,0,2)
error_prob_Astabs_fully_postselected_weight_4_err = np.array([0.5 * np.array([np.nanstd(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i],axis = -1)/np.sqrt(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i].shape[-1]) 
                                                                              for i in np.where(stab4A)[0]]).T for log in range(num_logicals)]).transpose(1,0,2)
error_prob_Bstabs_fully_postselected_weight_4_err  = np.array([0.5 * np.array([np.nanstd(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i],axis = -1)/np.sqrt(stabilizer_prod_per_round_per_stab_dataloss_postselection[log][i].shape[-1]) 
                                                                              for i in np.where(stab4B)[0]]).T for log in range(num_logicals)]).transpose(1,0,2)


error_prob_Astabs_postselected_withloss_weight_2  = 1-(1+np.abs(np.mean(stabilizer_prod_per_round_withloss[:,:,stab2A][:,:,:,perfect_reps],axis = -1)))/2
error_prob_Bstabs_postselected_withloss_weight_2  = 1-(1+np.abs(np.mean(stabilizer_prod_per_round_withloss[:,:,stab2B][:,:,:,perfect_reps], axis= -1)))/2
error_prob_Astabs_postselected_withloss_weight_4  = 1-(1+np.abs(np.mean(stabilizer_prod_per_round_withloss[:,:,stab4A][:,:,:,perfect_reps],axis = -1)))/2
error_prob_Bstabs_postselected_withloss_weight_4  = 1-(1+np.abs(np.mean(stabilizer_prod_per_round_withloss[:,:,stab4B][:,:,:,perfect_reps], axis= -1)))/2

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0,

In [ ]:
fig = plt.figure(figsize=(14,10))
fig.subplots_adjust(hspace = 0.4)

for log in range(num_logicals):  
    Astab_good_rounds_thislog = deterministic_rounds if prep_basis[log] == 'vertical' else nondeterministic_rounds
    Bstab_good_rounds_thislog = nondeterministic_rounds if prep_basis[log] == 'vertical' else deterministic_rounds
    
    plt.subplot(num_logicals,2,log*num_logicals + 1)

    plt.plot(np.arange(num_rounds)[Astab_good_rounds_thislog], error_prob_Astabs_postselected_weight_2[Astab_good_rounds_thislog, log],'o--',color='steelblue', alpha = 0.2, zorder = -20)
    plt.plot(np.arange(num_rounds)[Bstab_good_rounds_thislog], error_prob_Bstabs_postselected_weight_2[Bstab_good_rounds_thislog, log],'o--',color='steelblue', alpha = 0.2, zorder = -20)
    plt.plot(np.arange(num_rounds)[Astab_good_rounds_thislog], error_prob_Astabs_postselected_weight_4[Astab_good_rounds_thislog, log],'o--',color='firebrick', alpha = 0.2, zorder = -20)
    plt.plot(np.arange(num_rounds)[Bstab_good_rounds_thislog], error_prob_Bstabs_postselected_weight_4[Bstab_good_rounds_thislog, log],'o--',color='firebrick', alpha = 0.2, zorder = -20)
    plt.plot(np.arange(num_rounds)[Astab_good_rounds_thislog], np.mean(error_prob_Astabs_postselected_weight_2,axis=-1)[Astab_good_rounds_thislog, log],'o-',color='steelblue')
    plt.plot(np.arange(num_rounds)[Bstab_good_rounds_thislog], np.mean(error_prob_Bstabs_postselected_weight_2,axis=-1)[Bstab_good_rounds_thislog, log],'o-',color='steelblue', label='weight 2')
    plt.plot(np.arange(num_rounds)[Astab_good_rounds_thislog], np.mean(error_prob_Astabs_postselected_weight_4,axis=-1)[Astab_good_rounds_thislog, log],'o-',color='firebrick')
    plt.plot(np.arange(num_rounds)[Bstab_good_rounds_thislog], np.mean(error_prob_Bstabs_postselected_weight_4,axis=-1)[Bstab_good_rounds_thislog, log],'o-',color='firebrick', label='weight 4')
    plt.legend(frameon=False)
    plt.ylim(0, 0.5)
    plt.xticks([0,1,2])
    plt.xlabel('QEC cycle')
    plt.ylabel("Detection probability")
    
    all_good_weight2 = np.concatenate((error_prob_Astabs_postselected_weight_2[Astab_good_rounds_thislog, log], error_prob_Bstabs_postselected_weight_2[Bstab_good_rounds_thislog, log])).ravel()
    all_good_weight4 =  np.concatenate((error_prob_Astabs_postselected_weight_4[Astab_good_rounds_thislog, log], error_prob_Bstabs_postselected_weight_4[Bstab_good_rounds_thislog, log])).ravel()
    
    all_good_weight2_err = np.concatenate((error_prob_Astabs_postselected_weight_2_err[Astab_good_rounds_thislog, log], error_prob_Bstabs_postselected_weight_2_err[Bstab_good_rounds_thislog, log])).ravel()
    all_good_weight4_err = np.concatenate((error_prob_Astabs_postselected_weight_4_err[Astab_good_rounds_thislog, log], error_prob_Bstabs_postselected_weight_4_err[Bstab_good_rounds_thislog, log])).ravel()
    
    plt.title(f"""Logical {log+1}: Postselected on rearrangement only
    Weight 2: {np.mean(all_good_weight2):.4f}({np.mean(all_good_weight2_err)/np.sqrt(all_good_weight2.size):.4f}), Weight 4: {np.mean(all_good_weight4):.4f}({np.mean(all_good_weight4_err)/np.sqrt(all_good_weight4.size):.4f})
    Overall {np.mean(np.concatenate((all_good_weight2, all_good_weight4))):.4f}({np.mean(np.concatenate((all_good_weight2_err, all_good_weight4_err)))/np.sqrt(np.concatenate((all_good_weight2, all_good_weight4)).ravel().size):.4f})""")
    
    plt.subplot(num_logicals,2,log*num_logicals + 2)

    plt.plot(np.arange(num_rounds)[Astab_good_rounds_thislog], error_prob_Astabs_fully_postselected_weight_2[Astab_good_rounds_thislog, log],'o--',color='steelblue', alpha = 0.2, zorder = -20)
    plt.plot(np.arange(num_rounds)[Bstab_good_rounds_thislog], error_prob_Bstabs_fully_postselected_weight_2[Bstab_good_rounds_thislog, log],'o--',color='steelblue', alpha = 0.2, zorder = -20)
    plt.plot(np.arange(num_rounds)[Astab_good_rounds_thislog], error_prob_Astabs_fully_postselected_weight_4[Astab_good_rounds_thislog, log],'o--',color='firebrick', alpha = 0.2, zorder = -20)
    plt.plot(np.arange(num_rounds)[Bstab_good_rounds_thislog], error_prob_Bstabs_fully_postselected_weight_4[Bstab_good_rounds_thislog, log],'o--',color='firebrick', alpha = 0.2, zorder = -20)
    plt.plot(np.arange(num_rounds)[Astab_good_rounds_thislog], np.mean(error_prob_Astabs_fully_postselected_weight_2,axis=-1)[Astab_good_rounds_thislog, log],'o-',color='steelblue')
    plt.plot(np.arange(num_rounds)[Bstab_good_rounds_thislog], np.mean(error_prob_Bstabs_fully_postselected_weight_2,axis=-1)[Bstab_good_rounds_thislog, log],'o-',color='steelblue', label='weight 2')
    plt.plot(np.arange(num_rounds)[Astab_good_rounds_thislog], np.mean(error_prob_Astabs_fully_postselected_weight_4,axis=-1)[Astab_good_rounds_thislog, log],'o-',color='firebrick')
    plt.plot(np.arange(num_rounds)[Bstab_good_rounds_thislog], np.mean(error_prob_Bstabs_fully_postselected_weight_4,axis=-1)[Bstab_good_rounds_thislog, log],'o-',color='firebrick', label='weight 4')
    plt.legend(frameon=False)
    plt.ylim(0, 0.5)
    plt.xticks([0,1,2])
    plt.xlabel('QEC cycle')
    plt.ylabel("Detection probability")
    all_good_weight2 = np.concatenate((error_prob_Astabs_fully_postselected_weight_2[Astab_good_rounds_thislog, log], error_prob_Bstabs_fully_postselected_weight_2[Bstab_good_rounds_thislog, log])).ravel()
    all_good_weight4 =  np.concatenate((error_prob_Astabs_fully_postselected_weight_4[Astab_good_rounds_thislog, log], error_prob_Bstabs_fully_postselected_weight_4[Bstab_good_rounds_thislog, log])).ravel()
    
    all_good_weight2_err = np.concatenate((error_prob_Astabs_fully_postselected_weight_2_err[Astab_good_rounds_thislog, log], error_prob_Bstabs_fully_postselected_weight_2_err[Bstab_good_rounds_thislog, log])).ravel()
    all_good_weight4_err = np.concatenate((error_prob_Astabs_fully_postselected_weight_4_err[Astab_good_rounds_thislog, log], error_prob_Bstabs_fully_postselected_weight_4_err[Bstab_good_rounds_thislog, log])).ravel()
    
    plt.title(f"""Logical {log + 1}: Postselected on rearrangement and no data loss
    Weight 2: {np.mean(all_good_weight2):.4f}({np.mean(all_good_weight2_err)/np.sqrt(all_good_weight2.size):.4f}), Weight 4: {np.mean(all_good_weight4):.4f}({np.mean(all_good_weight4_err)/np.sqrt(all_good_weight4.size):.4f})
    Overall {np.mean(np.concatenate((all_good_weight2, all_good_weight4))):.4f}({np.mean(np.concatenate((all_good_weight2_err, all_good_weight4_err)))/np.sqrt(np.concatenate((all_good_weight2, all_good_weight4)).ravel().size):.4f})""")

In [ ]:
for log in np.arange(num_logicals):
    fig, axs = plt.subplots(ancilla_grid_size,ancilla_grid_size)
    # fig.canvas.set_window_title("Ancilla stabilizers, excluding loss")
    fig.subplots_adjust(wspace=0.3,hspace=0.3)
    count = 0
    for i in range(ancilla_grid_size**2):
        ax = axs[i//ancilla_grid_size,i%ancilla_grid_size]
        if ancilla_block_mask[i]:
            mean_stab = np.nanmean(stabilizer_prod_per_round[:,log,count,perfect_reps], axis = -1)
            # ax.bar(np.arange(num_rounds),mean_stab, color = ["r","k"][int(ancilla_Astabs_mask[count])])
            ax.bar(np.arange(num_rounds),mean_stab, color = ["r","k"][int(ancilla_Astabs_mask[count])])
            ax.set_xticks([])
            ax.set_ylim(-1,1)
            count+=1 
        else:
            ax.set_xticks([])
            ax.set_yticks([])
    fig.suptitle("Logical %i stab. products (perfect rearrangement, exclude loss)"%(log+1))
